# Litigation and A34(1) Refusal Trends in Canadian Immigration

## 1. Background and Datasets
We received two datasets from our partner organization, which were originally requested from IRCC:

- A34(1) Refusals: Covers temporary and permanent resident applications refused between Jan 1, 2019 – Dec 31, 2024, grouped by country of residence and citizenship.

- Litigation Applications: Covers litigation data from Jan 2018 – Dec 2023, broken down by case type, leave decision, country of citizenship, and processing office.

This narrative is based on the trends we found from both datasets. It tries to explore some of the key themes raised by our partner, such as the role of geopolitics, anti-Black racism, general trends in litigation, and what gaps still exist in the data.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
lit = pd.read_excel("../data/raw/litigation_cases.xlsx", skiprows=5, skipfooter=7)

In [3]:
ref = pd.read_csv('../data/processed/a34_1_refused_cleaned.csv')

## 2. Standardizing the Litigation Data
To make the litigation data easier to analyze, we standardized the column "LIT Leave Decision Desc":

- We grouped "Discontinued - Withdrawn at Leave" and "Discontinued - Consent at Leave" under Discontinued.

- "Allowed - Consent" was merged into Allowed.

These categories were quite similar in meaning, and merging them helped simplify the analysis, especially when comparing outcomes across countries.

In [4]:
# Standardizing Leave decision
lit['LIT Leave Decision Desc'] = lit['LIT Leave Decision Desc'].replace(
    to_replace=r'^Discontinued.*', value='Discontinued', regex=True
)

lit['LIT Leave Decision Desc'] = lit['LIT Leave Decision Desc'].replace(
    to_replace=r'^Dismissed.*', value='Dismissed', regex=True
)

lit['LIT Leave Decision Desc'] = lit['LIT Leave Decision Desc'].replace(
    to_replace=r'^Allowed.*', value='Allowed', regex=True
)

## 3. Countries Showing Up in Both Datasets
When we looked at the top countries in both litigation counts and A34(1) refusals, we found a few countries that showed up in both lists:

- Iran: 4,503 litigation cases and 70 A34(1) refusals

- India: 6,825 litigations and 32 refusals

- China: 3,223 litigations and 57 refusals

This suggests these countries are being scrutinized more, though that is just a hypothesis. We would need more data (like total application numbers or approval rates) to say anything with certainty.

At the same time, some countries only appear in one list:

- Nigeria had the highest number of litigation cases (7,819) but only 6 A34(1) refusals.

- Ukraine and Syria had very high A34(1) refusals (176 and 101), but they are not even in the top 10 for litigation counts.

In [5]:
top_countries = lit.groupby("Country of Citizenship")["LIT Litigation Count"].sum().nlargest(10).reset_index()
fig = px.bar(top_countries, x="Country of Citizenship", y="LIT Litigation Count",
             title="Top 10 Countries by Litigation Count", text_auto=True)
fig.show()

In [6]:
country_counts = ref.groupby("country")["count"].sum().nlargest(10).reset_index()
fig = px.bar(country_counts, x="country", y="count", title="Top 10 Countries by Total Refusals", text_auto=True)
fig.show()

## 4. Litigation Trends Over Time for Top 4 Countries
Looking at the top 4 countries by litigation count — Nigeria, India, Iran, and China — we broke down litigation applications by case type over time (2018–2023). Here’s what we found:

- Iran barely had any litigation before 2020, but applications picked up in 2021. Around 90% of them are Visa Officer Refusals.

- Nigeria had a very different pattern, about 80% of its cases are RAD decisions, with the number peaking in 2021 and then going down.

- India and China have a similar RAD case distribution, but China had a spike in Mandamus applications in 2023.

- India shows a steady increase in both Visa Officer Refusals and RAD decisions over the years.

In [16]:
countries = {
    "People's Republic of China": "China",
    "India": "India",
    "Iran": "Iran",
    "Nigeria": "Nigeria"
}
valid_case_types = ["RAD Decisions", "Visa Officer Refusal", "Mandamus"]

# Use pastel colors
color_palette = px.colors.qualitative.Pastel
color_map = dict(zip(valid_case_types, color_palette[:len(valid_case_types)]))

# Prepare subplot: 2 rows (summary + bar) and N columns (per country)
fig = make_subplots(
    rows=2, cols=4,
    shared_xaxes=False,
    shared_yaxes=True,
    vertical_spacing=0.1,
    horizontal_spacing=0.03,
    subplot_titles=list(countries.values()),
    row_heights=[0.2, 0.8]
)

# Loop through each country and populate subplot
for col_idx, (country_key, country_name) in enumerate(countries.items(), start=1):
    df_country = lit[lit["Country of Citizenship"] == country_key]
    df_country = df_country[df_country["LIT Case Type Group Desc"].isin(valid_case_types)]

    grouped = df_country.groupby(
        ["LIT Leave Decision Date - Year", "LIT Case Type Group Desc"]
    )["LIT Litigation Count"].sum().reset_index()

    pivot_df = grouped.pivot(
        index="LIT Leave Decision Date - Year",
        columns="LIT Case Type Group Desc",
        values="LIT Litigation Count"
    ).fillna(0).sort_index()

    total_counts = pivot_df.sum()
    total_percent = (total_counts / total_counts.sum() * 100).round(2)

    # --- Row 1: Summary bar ---
    for case_type in valid_case_types:
        fig.add_trace(go.Bar(
            y=["Total"],
            x=[total_percent.get(case_type, 0)],
            name=case_type,
            orientation='h',
            text=[f"{total_percent.get(case_type, 0)}%"],
            textposition='outside',
            textfont=dict(color='black'),
            marker=dict(color=color_map[case_type]),
            showlegend=(col_idx == 1)  # Show legend only in first column
        ), row=1, col=col_idx)

    # --- Row 2: Per-year stacked bars ---
    for case_type in valid_case_types:
        fig.add_trace(go.Bar(
            y=pivot_df.index.astype(str),
            x=pivot_df[case_type],
            name=case_type,
            orientation='h',
            text=pivot_df[case_type],
            textposition='outside',
            textfont=dict(color='black'),
            marker=dict(color=color_map[case_type]),
            showlegend=False  # Avoid legend clutter
        ), row=2, col=col_idx)

    # Hide x-axes
    fig.update_xaxes(visible=False, row=1, col=col_idx)
    fig.update_xaxes(visible=False, row=2, col=col_idx)

    # Reverse y-axis for second row
    fig.update_yaxes(autorange='reversed', row=2, col=col_idx)
    fig.update_yaxes(title='', row=1, col=col_idx)
    if col_idx != 1:
        fig.update_yaxes(showticklabels=False, row=2, col=col_idx)  # Only leftmost y-axis visible

# Layout adjustments
fig.update_layout(
    height=900,
    width=2000,
    barmode='stack',
    plot_bgcolor='white',
    title_x=0.5,
    font=dict(size=20),
    legend=dict(
        title="Case Types",
        orientation="h",
        yanchor="bottom",
        y=-0.1,
        xanchor="center",
        x=0.5,
        font=dict(size=20)
    )
)

fig.show()

## 5. How Do Litigation Outcomes Vary by Country?
Here is the overall breakdown across all litigation applications:

- Dismissed: 51.48%

- Discontinued: 28.87%

- Allowed: 19.66%

Now comparing specific countries to these averages:

- Iran has 20% fewer dismissals than average, but 28% more discontinued cases. Its allowed rate is also 8.7% lower than average.

- Nigeria has a 9% higher dismissal rate and 8% fewer discontinued cases. Allowed rate is about the same.

- India has a higher discontinued rate (+7%) and lower allowed rate (–8%), but similar dismissal rate.

- China is pretty much aligned with the overall averages for all decision types.

In [14]:
decision_desc = lit.groupby("LIT Leave Decision Desc")["LIT Litigation Count"].sum().nlargest(3).reset_index()

# Calculate total for annotations
total_litigation = decision_desc["LIT Litigation Count"].sum()

# Add a percentage column for clarity
decision_desc["Percentage"] = (decision_desc["LIT Litigation Count"] / total_litigation * 100).round(2)

grouped_df = (
    lit.groupby(["Country of Citizenship", "LIT Leave Decision Desc"])["LIT Litigation Count"]
    .sum()
    .reset_index()
)

# Find top 5 countries by total litigation count
top_countries = (
    grouped_df.groupby("Country of Citizenship")["LIT Litigation Count"]
    .sum()
    .nlargest(4)
    .index
)

# Filter data for top 5 countries
filtered_df = grouped_df[grouped_df["Country of Citizenship"].isin(top_countries)]

# Calculate percentage contribution within each country
# Calculate percentage contribution within each country
filtered_df["Percentage"] = (
    filtered_df.groupby("Country of Citizenship")["LIT Litigation Count"]
    .transform(lambda x: (x / x.sum() * 100) if x.sum() > 0 else 0)  # Avoid division by zero
    .round(2)  # Round to 2 decimal places
)


# Prepare table data
table_data = filtered_df.sort_values(
    ["Country of Citizenship", "LIT Litigation Count"], ascending=[True, False]
)
dumble_data = table_data[
    ~table_data["LIT Leave Decision Desc"].isin(['Not Started at Leave', 'No Leave Required', 'Leave Exception'])
]
decision_desc

/var/folders/f9/98mwc4pn4l51n7xqzs0w7ryh0000gn/T/ipykernel_40455/1553467676.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,LIT Leave Decision Desc,LIT Litigation Count,Percentage
0,Dismissed,24843,51.48
1,Discontinued,13930,28.87
2,Allowed,9486,19.66


In [ ]:
total_df = decision_desc[['LIT Leave Decision Desc', 'Percentage']].rename(columns={'Percentage': 'Total_Percentage'})

# country_df: country percentages per decision type
country_df = dumble_data[['Country of Citizenship', 'LIT Leave Decision Desc', 'Percentage']]

# Merge and compute difference
merged = pd.merge(country_df, total_df, on='LIT Leave Decision Desc')
merged['Difference'] = merged['Percentage'] - merged['Total_Percentage']

fig = go.Figure()

# Show legend only for countries that appear in 'Dismissed'
dismissed_countries = set(merged[merged['LIT Leave Decision Desc'] == 'Dismissed']['Country of Citizenship'])

# Color map (optional)
unique_countries = merged['Country of Citizenship'].unique()
color_map = {country: f"hsl({i * 60 % 360}, 70%, 50%)" for i, country in enumerate(unique_countries)}

for decision in merged['LIT Leave Decision Desc'].unique():
    subset = merged[merged['LIT Leave Decision Desc'] == decision]

    for i, (_, row) in enumerate(subset.iterrows()):
        fig.add_trace(go.Scatter(
            x=[0, row['Difference']],
            y=[decision, decision],
            mode='lines',
            line=dict(color='gray', width=2),
            showlegend=False
        ))

        text_pos = 'top center' if i % 2 == 0 else 'bottom center'
        show_legend_label = (row['Country of Citizenship'] in dismissed_countries) and (decision == 'Dismissed')

        fig.add_trace(go.Scatter(
            x=[row['Difference']],
            y=[decision],
            mode='markers',
            marker=dict(size=16, color=color_map[row['Country of Citizenship']], symbol='circle'),
            showlegend=show_legend_label,
            name=row['Country of Citizenship'],
            hovertemplate=(
                f"{row['Country of Citizenship']}<br>"
                f"Decision: {decision}<br>"
                f"Difference: {row['Difference']:.2f}%<extra></extra>"
            )
        ))

        fig.add_annotation(
            x=row['Difference'],
            y=decision,
            text=f"{row['Difference']:.2f}%",
            showarrow=False,
            font=dict(size=14, color='white'),
            align='center',
            bgcolor=color_map[row['Country of Citizenship']],
            borderpad=4,
            yshift=12 if i % 2 == 0 else -12
        )


fig.update_layout(
    xaxis=dict(title="Difference in Percentage (country % - total %)"),
    yaxis=dict(title="Leave Decision", autorange='reversed', gridcolor='white'),
    height=800,
    width=1500,
    plot_bgcolor='white',
    font=dict(family='Arial, sans-serif', size=20),
    hovermode="closest",
    legend=dict(
        orientation="h",
        y=-0.3,
        x=0.5,
        xanchor="center",
        bordercolor="white",
        borderwidth=1,
        itemclick="toggle",
        itemdoubleclick="toggleothers"
    )
)

fig.show()

## 6. Decision Type by Case Type
We also broke down outcomes by case type, and there are some clear patterns:

- Mandamus has a much higher discontinuation rate, about 40% above average.

- Visa Officer Refusals are also discontinued more (+27.6%).

- RAD cases, on the other hand, are discontinued far less (–21%).

- For dismissals, RAD is 18% higher than average, while mandamus and visa refusals are both lower by 25% and 19% respectively.

- Allowed rates are lowest for mandamus (–15%) and visa refusals (–8%). RAD cases are close to average (+2%).

In [19]:
grouped_df = (
    lit.groupby(["LIT Case Type Group Desc", "LIT Leave Decision Desc"])["LIT Litigation Count"]
    .sum()
    .reset_index()
)

specific_case_types = ["RAD Decisions", "Visa Officer Refusal", "Mandamus"]

# Filter data for these specific case types only
filtered_df = grouped_df[grouped_df["LIT Case Type Group Desc"].isin(specific_case_types)]

# Calculate percentage contribution within each case type
filtered_df["Percentage"] = (
    filtered_df.groupby("LIT Case Type Group Desc")["LIT Litigation Count"]
    .transform(lambda x: (x / x.sum() * 100) if x.sum() > 0 else 0)
    .round(2)
)

# Prepare table data sorted by case type and litigation count
table_data = filtered_df.sort_values(
    ["LIT Case Type Group Desc", "LIT Litigation Count"], ascending=[True, False]
)

# Filter out unwanted leave decisions (same as your dumble_data1 logic)
dumble_data1 = table_data[
    ~table_data["LIT Leave Decision Desc"].isin(['Not Started at Leave', 'No Leave Required', 'Leave Exception'])
]

/var/folders/f9/98mwc4pn4l51n7xqzs0w7ryh0000gn/T/ipykernel_40455/3935929983.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
total_df = decision_desc[['LIT Leave Decision Desc', 'Percentage']].rename(columns={'Percentage':'Total_Percentage'})

case_type_df = dumble_data1[['LIT Case Type Group Desc', 'LIT Leave Decision Desc', 'Percentage']]

merged = pd.merge(case_type_df, total_df, on='LIT Leave Decision Desc')
merged['Difference'] = merged['Percentage'] - merged['Total_Percentage']

fig = go.Figure()

dismissed_case_types = set(merged[merged['LIT Leave Decision Desc'] == 'Dismissed']['LIT Case Type Group Desc'])

unique_case_types = merged['LIT Case Type Group Desc'].unique()
color_map = {case_type: f"hsl({i * 60 % 360}, 70%, 50%)" for i, case_type in enumerate(unique_case_types)}

for decision in merged['LIT Leave Decision Desc'].unique():
    subset = merged[merged['LIT Leave Decision Desc'] == decision]
    
    for i, (_, row) in enumerate(subset.iterrows()):
        # Line from 0 to difference
        fig.add_trace(go.Scatter(
            x=[0, row['Difference']],
            y=[decision, decision],
            mode='lines',
            line=dict(color='gray', width=2),
            showlegend=False
        ))

        # Marker
        show_legend_flag = (row['LIT Case Type Group Desc'] in dismissed_case_types) and (decision == 'Dismissed')
        
        fig.add_trace(go.Scatter(
            x=[row['Difference']],
            y=[decision],
            mode='markers',
            marker=dict(size=16, color=color_map[row['LIT Case Type Group Desc']], symbol='circle'),
            showlegend=show_legend_flag,
            name=row['LIT Case Type Group Desc'],
            hovertemplate=(
                f"Case Type Group: {row['LIT Case Type Group Desc']}<br>"
                f"Decision: {decision}<br>"
                f"Difference: {row['Difference']:.2f}%<extra></extra>"
            )
        ))

        # Annotated colored percentage label
        fig.add_annotation(
            x=row['Difference'],
            y=decision,
            text=f"{row['Difference']:.2f}%",
            showarrow=False,
            font=dict(size=14, color='white'),
            align='center',
            bgcolor=color_map[row['LIT Case Type Group Desc']],
            borderpad=4,
            yshift=12 if i % 2 == 0 else -12  # alternate text positions
        )

fig.update_layout(
    xaxis=dict(
        title="Difference in Percentage (case type % - total %)",
    ),
    yaxis=dict(
        title="Leave Decision Description",
        autorange='reversed',
        gridcolor='white'
    ),
    height=800,
    width=1500,
    plot_bgcolor='white',
    font=dict(family='Arial, sans-serif', size=20),
    hovermode="closest",
    legend=dict(
        orientation="h",
        y=-0.3,
        x=0.5,
        xanchor="center",
        bordercolor="white",
        borderwidth=1,
        itemclick="toggle",
        itemdoubleclick="toggleothers"
    )
)

fig.show()

## 7. A34(1) Refusals – Country Patterns
In the refusal dataset, a few things stood out:

- Ukraine had a massive spike in 2024 — 131 refusals, compared to just 5 in 2023.

- Syria had the most refusals in 2019 and 2022.

- China peaked in 2024, while Bangladesh spiked in 2019.

- Iran’s refusals have been fairly steady since 2022.

But most countries have relatively low A34(1) refusal counts overall, so we can’t read too much into year-to-year changes, except for Ukraine, where the 2024 jump is clearly significant.

In [11]:
country_counts = ref.groupby('country')['count'].sum().sort_values(ascending=False)

# --- 2. Get top 5 countries by total count ---
top_countries = country_counts.head(5).index.tolist()

# --- 3. Filter original dataframe for those countries ---
df_top = ref[ref['country'].isin(top_countries)]

# --- 4. Create pivot table for heatmap ---
heatmap_data = df_top.pivot_table(index='country', columns='year', values='count', aggfunc='sum', fill_value=0)

# --- 5. Create heatmap using Plotly ---
fig = px.imshow(
    heatmap_data,
    text_auto=True,
    color_continuous_scale='Blues',
    aspect='auto',
    labels=dict(x="Year", y="Country", color="Count")
)

fig.update_layout(
    title_x=0.5,
    font=dict(size=14),
    xaxis=dict(side='top')
)

fig.show()

## 8. A34(1) Grounds – What is Behind the Refusals?

To better understand the nature of A34(1) refusals, we broke down the specific inadmissibility grounds for the top five countries with the highest refusal counts. Here is what stood out:

- Ukraine: Around 80% of the refusals fall under A34(1)(f), with the rest mostly under (a). Interestingly, almost all of these refusals are for temporary resident applications.

- Bangladesh: About 90% of refusals are based on A34(1)(f), and like Syria, most refusals are for permanent residents.

- Iran: Refusals are more evenly distributed across multiple grounds—A34(1)(f), (d), (c), and (b)—suggesting a broader application of inadmissibility clauses in their case.

- Syria: Around 50% of refusals fall under (f), but there is also a significant number under (d), (b), and (c). These are again mostly permanent resident cases.

- China: Roughly 70% are based on A34(1)(f), with smaller shares under (a) and (c).

In [15]:
fig_top10 = px.treemap(
    df_top,
    path=["country", "inadmissibility_grounds"],
    values="count",
    hover_data=["count"],
)

fig_top10.update_traces(
    textinfo="label+value+percent entry"
)

fig_top10.update_layout(
    width=1400,
    height=800,
    margin=dict(t=50, l=5, r=5, b=5),
    font=dict(
        size=20
    ),
    title_font_size=24
)

fig_top10.show()

## 9. What Does All This Suggest?
### Geopolitics
There seems to be some connection between international relations and immigration outcomes. For example, Ukraine’s sudden spike in refusals in 2024 and Iran’s consistently high litigation volume could be influenced by broader geopolitical dynamics. But we can't confirm this without better data.

### Anti-Black Racism
Nigeria's numbers are interesting, it's the top country for litigation but barely shows up in A34(1) refusals. This might suggest a different kind of bias or structural issue, especially when we consider the high rate of RAD litigation. We would need more disaggregated data to explore this further.

### Litigation Trends
- Mandamus cases are on the rise — especially from China in 2023.

- Visa refusals are a major litigation trigger for Iran.

- RAD cases are common for African countries, especially Nigeria.

### Data Gaps
A lot of what we would like to say more confidently is blocked by missing data:

- We don't have the filing date for litigation or the original application date.

- There is no link between a refusal and any corresponding litigation.

- And we don’t know how many applications were approved or refused by country, we only see the refusals.

These gaps mean we can’t draw strong conclusions or do statistical significance testing. The data is interesting, but not transparent or complete enough.

## 10. Final Thoughts
We have uncovered some clear patterns, like spikes in litigation and refusal for specific countries, or how case types affect outcomes. But the bigger story is that the current datasets don’t let us connect the dots all the way. Until IRCC and the courts release more detailed and linked data, we’ll keep running into these same limitations.